# MLP
A notebook about MLP

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Tensorflow is required
import tensorflow as tf

# Common imports
import copy
import pandas as pd
import numpy as np
import os
PRJ_ROOT_DIR = os.path.dirname(os.path.abspath(''))

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import seaborn as sns

In [2]:
# Where to save the figures
NOTE_ROOT_DIR = os.path.abspath('')
DATA_DIR = os.path.join(NOTE_ROOT_DIR, "data", "20news-bydate")
CHAPTER_ID = "01_mlp"
IMAGES_PATH = os.path.join(NOTE_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## 1 - Load the Data

In [3]:
# Setup data reader (so-called data loader)
from models import DataReader
with open(os.path.join(DATA_DIR, "words_idfs.txt")) as f:
    vocab_size = len(f.read().splitlines())

def load_dataset():
    test_data_reader = DataReader(
        data_path=os.path.join(DATA_DIR, "data_tf_idf.txt"),
        batch_size=50,
        vocab_size=vocab_size,
        size=(0.8, 1)
    )
    train_data_reader = DataReader(
        data_path=os.path.join(DATA_DIR, "data_tf_idf.txt"),
        batch_size=50,
        vocab_size=vocab_size
    )
    return train_data_reader, test_data_reader

## 2 - MLP

### Pre-setup the Utilities

In [14]:
# Save the parameters
def save_parameters(name, value, epoch):
    filename = name.replace(":", '-colon-') + '-epoch-{}.txt'.format(epoch)
    if len(value.shape) == 1:
        string_form = ",".join([str(number) for number in value])
    else:
        string_form = '\n'.join([",".join([str(number) for number in value[row]]) for row in range(value.shape[0])])
    with open(os.path.join(NOTE_ROOT_DIR, "data", "saved-paras", filename), "w") as f:
        f.write(string_form)

In [10]:
# Restore the parameters
def restore_parameter(name, epoch):
    filename = name.replace(":", '-colon-') + '-epoch-{}.txt'.format(epoch)
    with open(os.path.join(NOTE_ROOT_DIR, "data", "saved-paras", filename)) as f:
        lines = f.read().splitlines()
    if len(lines) == 1:
        value = [float(number) for number in lines[0].split(",")]
    else:
        value = [[float(number) for number in lines[row].split(",")] for row in range(len(lines))]
    return value

### Training the Model

In [6]:
# Build computation graph
from models import MLP
mlp = MLP(vocab_size=vocab_size, hidden_size=50)
pred_y, loss = mlp.build_graph()
train_op = mlp.trainer(loss=loss, learning_rate=0.1)

In [19]:
# Open a session to train the model
with tf.compat.v1.Session() as sess:
    train_data_reader, test_data_reader = load_dataset()
    step, MAX_STEP = 0, 10000
    
    sess.run(tf.compat.v1.global_variables_initializer())
    while step < MAX_STEP:
        train_data, train_labels = train_data_reader.next_batch()
        plabels_eval, loss_eval, _ = sess.run(
            [pred_y, loss, train_op],
            feed_dict={
                mlp._X: train_data,
                mlp._real_Y: train_labels
            }
        )
        step += 1
        if step % 100 == 0:
            print("Step: {}, loss: {}".format(step, loss_eval))
        
    trainable_variables = tf.compat.v1.trainable_variables()
    for variable in trainable_variables:
        save_parameters(
            name=variable.name,
            value=variable.eval(),
            epoch=train_data_reader._num_epoch
        )

Step: 100, loss: 0.0
Step: 200, loss: 0.0
Step: 300, loss: 0.0
Step: 400, loss: 0.009043372236192226
Step: 500, loss: 4.296870974940248e-05
Step: 600, loss: 1.3999577276990749e-05
Step: 700, loss: 5.72202543480671e-06
Step: 800, loss: 7.326543709496036e-06
Step: 900, loss: 7.779447514622007e-06
Step: 1000, loss: 9.196145401801914e-05
Step: 1100, loss: 4.005397897799412e-07
Step: 1200, loss: 4.29153317327291e-08
Step: 1300, loss: 1.311299229200813e-07
Step: 1400, loss: 7.210942385427188e-06
Step: 1500, loss: 3.5545549508242402e-06
Step: 1600, loss: 8.081491978373379e-05
Step: 1700, loss: 1.5974009670571832e-07
Step: 1800, loss: 1.192092646817855e-08
Step: 1900, loss: 1.430510998545742e-08
Step: 2000, loss: 3.337858700547258e-08
Step: 2100, loss: 4.124606789446261e-07
Step: 2200, loss: 5.555075404117815e-07
Step: 2300, loss: 0.0
Step: 2400, loss: 0.0
Step: 2500, loss: 0.0
Step: 2600, loss: 0.0
Step: 2700, loss: 0.0
Step: 2800, loss: 0.0
Step: 2900, loss: 0.0
Step: 3000, loss: 0.0
Step: 3

### Test the Model

In [21]:
# Open a session to test the model
with tf.compat.v1.Session() as sess:
    _, test_data_reader = load_dataset()
    step, MAX_STEP = 0, 3000
    
    with tf.compat.v1.Session() as sess:
        epoch = 0
        trainable_variables = tf.compat.v1.trainable_variables()
        for variable in trainable_variables:
            saved_value = restore_parameter(
                name=variable.name,
                epoch=epoch
            )
            assign_op = variable.assign(saved_value)
            sess.run(assign_op)
        
        num_true_preds = 0
        while step < MAX_STEP:
            test_data, test_labels = test_data_reader.next_batch()
            test_plabels_eval = sess.run(
                pred_y,
                feed_dict={
                    mlp._X: test_data,
                    mlp._real_Y: test_labels
                }
            )
            matches = np.equal(test_plabels_eval, test_labels)
            num_true_preds += np.sum(matches.astype("float"))
            
            step += 1
            if step % 100 == 0:
                print(f"Test on the batch {step}")

            if test_data_reader._batch_id == 0:
                break
        
        print("========")
        print("Epoch:", epoch)
        print("Accuracy on the test data:", num_true_preds / len(test_data_reader._data))

Test on the batch 100
Test on the batch 200
Test on the batch 300
Test on the batch 400
Test on the batch 500
Test on the batch 600
Test on the batch 700
Test on the batch 800
Test on the batch 900
Test on the batch 1000
Test on the batch 1100
Test on the batch 1200
Test on the batch 1300
Test on the batch 1400
Test on the batch 1500
Test on the batch 1600
Test on the batch 1700
Test on the batch 1800
Test on the batch 1900
Test on the batch 2000
Test on the batch 2100
Test on the batch 2200
Test on the batch 2300
Test on the batch 2400
Test on the batch 2500
Test on the batch 2600
Test on the batch 2700
Test on the batch 2800
Test on the batch 2900
Test on the batch 3000
Epoch: 0
Accuracy on the test data: 0.2509283819628647
